In [1]:
# Análise e manipulação de dados
import numpy as np
import pandas as pd
# Dados séries temporai
from datetime import datetime, timedelta
import pytz
# Integração com MetaTrader 5
import datetime as dt
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")


In [2]:
mt5.initialize()

True

In [3]:
def baixar_dados(ticker):
    cotacoes_mt5_all = []

    timezone = pytz.timezone("America/Sao_Paulo")
    data_inicio = datetime.now() - timedelta(days=600)
    data_fim = datetime.now()

    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_H1, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)
    return pd.concat(cotacoes_mt5_all, axis=0)


In [4]:
def tratamento_df(df_final):
    df_final = df_final.drop('time', axis =1)

    df_final = df_final.reset_index()

    # Criar a coluna "data" contendo apenas a parte da data
    df_final['data'] = df_final['time'].dt.date

    # Criar a coluna "hora" contendo apenas a parte da hora
    df_final['hora'] = df_final['time'].dt.time

    df_final = df_final.drop('time', axis =1)
    return df_final

In [5]:
def window_func(df_final):
    # Sort the DataFrame by 'data' and 'ticker', then by 'hora'
    df_final.sort_values(by=['data', 'ticker', 'hora'], inplace=True)

    # Apply the window function and create a new column 'numeration'
    df_final['numeration'] = df_final.groupby(['data', 'ticker']).cumcount() + 1
        
    # Find the maximum 'numeration' for each day and ticker
    max_numeration_per_day = df_final.groupby(['data', 'ticker'])['numeration'].max()

    # Merge the result back into the original DataFrame to get the closing value (close) for each day
    df_final = df_final.merge(max_numeration_per_day, on=['data', 'ticker'], suffixes=('', '_max'))

    closing_values_per_day = df_final[df_final['numeration'] == df_final['numeration_max']][['data', 'ticker', 'close']]

    df_final = df_final.merge(closing_values_per_day, on=['data', 'ticker'], how='left')

    df_final.rename(columns={'close_y': 'fechamento_do_dia'}, inplace=True)
    #df_final.drop(['numeration','numeration_max'], inplace=True, axis=1)
    return df_final

In [6]:
def dia_anterior(df):
    df_anterior = df[['data', 'fechamento_do_dia']].groupby('data').last().reset_index()
    df_anterior.columns = ['data', 'fechamento_dia_anterior']

    df_anterior['fechamento_dia_anterior'] = df_anterior['fechamento_dia_anterior'].shift(1)

    # Faça um left join com base na coluna 'data'
    df_resultado = pd.merge(df, df_anterior, on='data', how='left')
    return df_resultado

In [7]:
def create_perc(df_ativo, percentual):
    #Ajustar variáveis
    df_ativo['entrada'] = df_ativo['fechamento_dia_anterior'] + (df_ativo['fechamento_dia_anterior'] * percentual)
    df_ativo['percentual'] = percentual
    return df_ativo

In [8]:
def create_stats (df_ativo, dias_analisados, horario):
    df_ativo['numeration'] = df_ativo['numeration'].astype(int)
    df_ativo = df_ativo.loc[df_ativo['numeration'] == horario]

    df_ativo['tem_entrada'] = np.where(df_ativo['open'] > df_ativo['entrada'],1,0)
    df_ativo['alvo'] = np.where(df_ativo['tem_entrada'] == 1 , ((df_ativo['fechamento_do_dia'] / df_ativo['open'] -1) *100)*-1, 0) 

    df_filtrado = df_ativo.loc[df_ativo['tem_entrada']==1]
    df_filtrado = df_filtrado.tail(dias_analisados)
    df_filtrado['alvo_acumulado'] = df_filtrado['alvo'].cumsum()
    return df_filtrado

In [9]:
def calcular_drawdown(df_filtrado, horario):
    df_filtrado['max_acumulado'] = df_filtrado['alvo_acumulado'].cummax()
    df_filtrado['drawdown'] = df_filtrado['alvo_acumulado'] - df_filtrado['max_acumulado']

    # Calcular o drawdown máximo
    drawdown_maximo = df_filtrado['drawdown'].min()

    # Localizar o índice do valor mínimo em 'drawdown'
    indice_drawdown_maximo = df_filtrado['drawdown'].idxmin()

    # Acessar a data correspondente ao índice do drawdown máximo
    data_drawdown_maximo = df_filtrado.loc[indice_drawdown_maximo, 'data']

    # Calcular a taxa de acerto
    resultado = df_filtrado['alvo'] > 0
    taxa_acerto = sum(resultado) / len(df_filtrado['alvo'])

    # Separar resultados positivos e negativos
    resultados_positivos = df_filtrado.loc[df_filtrado['alvo'] > 0, 'alvo']
    resultados_negativos = df_filtrado.loc[df_filtrado['alvo'] <= 0, 'alvo']

    try:
        pior_dia = min(resultados_negativos)
        melhor_dia = max(resultados_positivos)
    except:
        pior_dia = 0
        melhor_dia = 0
        
    indice_pior_dia = df_filtrado['alvo'].idxmin()
    data_pior_dia = df_filtrado.loc[indice_pior_dia, 'data']
    
    quantidade_trades = len(resultados_positivos) + len(resultados_negativos)
    # Calcular médias
    media_positivos = resultados_positivos.mean()
    media_negativos = resultados_negativos.mean()
    media_geral = df_filtrado['alvo'].mean()

    # Calcular frequência de resultados positivos e negativos
    freq_positivos = len(resultados_positivos) / len(df_filtrado)
    freq_negativos = len(resultados_negativos) / len(df_filtrado)

    # Calcular expectativa matemática
    expectativa = freq_positivos * media_positivos - freq_negativos * abs(media_negativos)

    # Calcular retorno médio diário
    retorno_medio_diario = df_filtrado['alvo'].mean()

    # Calcular desvio padrão diário
    desvio_padrao_diario = df_filtrado['alvo'].std()

    # Calcular índice de Sharpe
    indice_sharpe = np.sqrt(252) * retorno_medio_diario / desvio_padrao_diario
    
    ativo =  df_filtrado.iloc[2]['ticker']
    retorno_acumulado = df_filtrado.iloc[-1]['alvo_acumulado']
    primeiro_dia = df_filtrado.iloc[1]['data']
    
    resultado_dict = {
        'ativo': ativo,
        'horário': horario,
        'Retorno Acumulado': retorno_acumulado,
        'Melhor dia': melhor_dia,           
        'Pior Dia':pior_dia, 
        'Data Pior dia': data_pior_dia,
        'Primeiro Dia': primeiro_dia,
        'Quantidade de Trades': quantidade_trades,
        'Taxa de Acerto': taxa_acerto,
        'Media Geral': media_geral,
        'Média Positivos': media_positivos,
        'Média Negativos': media_negativos,
        'Expectativa Matemática': expectativa,
        'Índice de Sharpe': indice_sharpe,
        'Drawdown Máximo': drawdown_maximo,
        'Data do Drawdown': data_drawdown_maximo
    }

    return resultado_dict

In [10]:
ibov = ['SEQL3',
'AERI3',
'WEST3',
'TRAD3',
'KRSA3',
'ESPA3',
'IFCM3',
'NGRD3',
'VIIA3',
'MEAL3',
'SGPS3',
'BMGB4',
'LPSB3',
'PDTC3',
'SHOW3',
'CVCB3',
'LUPA3',
'MGLU3',
'UCAS3',
'BRIT3',
'MLAS3',
'COGN3',
'TECN3',
'PINE4',
'NINJ3',
'TCSA3',
'MBLY3',
'HBSA3',
'RAIZ4',
'CIEL3',
'HBOR3',
'CMIN3',
'SYNE3',
'PGMN3',
'QUAL3',
'PFRM3',
'ZAMP3',
'FIQE3',
'OSXB3',
'CSED3',
'POMO3',
'ANIM3',
'HAPV3',
'DMVF3',
'MELK3',
'AGXY3',
'ALPK3',
'CBAV3',
'CLSA3',
'OPCT3',
'PDGR3',
'LVTC3',
'JHSF3',
'RDNI3',
'POMO4',
'HBRE3',
'GFSA3',
'CEAB3',
'SEER3',
'TRIS3',
'LJQQ3',
'WIZC3',
'GUAR3',
'ALLD3',
'AVLL3',
'PETZ3',
'USIM5',
'LWSA3',
'USIM3',
'GRND3',
'MTRE3',
'CAML3',
'GMAT3',
'BOAS3',
'EVEN3',
'MRFG3',
'JALL3',
'PTBL3',
'ECOR3',
'CPLE6',
'BPAN4',
'POSI3',
'DEXP4',
'LAVV3',
'ALPA4',
'DXCO3',
'GOLL4',
'CASH3',
'DEXP3',
'ITSA4',
'ITSA3',
'STBP3',
'RAPT3',
'KEPL3',
'ALPA3',
'BRFS3',
'JSLG3',
'MATD3',
'BEEF3',
'CXSE3',
'RANI3',
'SIMH3',
'PLPL3',
'SOMA3',
'AESB3',
'MEGA3',
'MDNE3',
'ONCO3',
'BMEB4',
'SOJA3',
'CRFB3',
'VITT3',
'VAMO3',
'PRNR3',
'CEDO4',
'MOVI3',
'CMIG4',
'ODPV3',
'RAPT4',
'TTEN3',
'MILS3',
'FHER3',
'ENEV3',
'CCRO3',
'PNVL3',
'SBFG3',
'GOAU3',
'EUCA4',
'BMEB3',
'PORT3',
'TEND3',
'GOAU4',
'FRAS3',
'DASA3',
'ASAI3',
'AURE3',
'MRVE3',
'BMOB3',
'BBDC3',
'TFCO4',
'BRSR3',
'ARML3',
'ENAT3',
'ROMI3',
'BRSR6',
'DESK3',
'TIMS3',
'MYPK3',
'B3SA3',
'ELMD3',
'CSUD3',
'BBDC4',
'TASA4',
'TASA3',
'FLRY3',
'BRBI11',
'VLID3',
'VBBR3',
'AZUL4',
'EUCA3',
'CURY3',
'EMBR3',
'NTCO3',
'CSMG3',
'GGPS3',
'CMIG3',
'UGPA3',
'NEOE3',
'ABCB4',
'LREN3',
'CSAN3',
'JBSS3',
'PCAR3',
'LOGG3',
'VVEO3',
'VULC3',
'SAPR11',
'BLAU3',
'YDUQ3',
'BRAP3',
'DIRR3',
'IGTI11',
'AMBP3',
'RECV3',
'EZTC3',
'RAIL3',
'BRAP4',
'KLBN11',
'BRKM5',
'INTB3',
'SCAR3',
'SMFT3',
'ALSO3',
'TGMA3',
'AALR3',
'ITUB3',
'BRKM3',
'OFSA3',
'CYRE3',
'LAND3',
'TRPL4',
'GGBR3',
'AGRO3',
'SQIA3',
'MULT3',
'PSSA3',
'RADL3',
'ITUB4',
'GGBR4',
'TUPY3',
'ALUP11',
'TOTS3',
'PETR4',
'VIVA3',
'APER3',
'BBSE3',
'EQTL3',
'PETR3',
'BPAC11',
'SMTO3',
'TRPL3',
'RDOR3',
'CPFE3',
'ELET3',
'TAEE11',
'RRRP3',
'ORVR3',
'IRBR3',
'WEGE3',
'ELET6',
'MDIA3',
'EGIE3',
'HYPE3',
'SLCE3',
'LEVE3',
'BBAS3',
'PRIO3',
'LOGN3',
'ENGI11',
'SUZB3',
'FESA4',
'SBSP3',
'CLSC3',
'CLSC4',
'RENT3',
'VALE3',
'ARZZ3']

In [11]:
horarios = [1,2,3,4,5,6,7,8,9]
lista_resultados = []
for ativo in ibov:
    print( f'Baixando os dados de {ativo}')
    for hora in horarios:
        try:
            df_final = baixar_dados(ativo)
            df_final = tratamento_df(df_final)
            df_final = window_func(df_final)
            df_final = dia_anterior(df_final)
            df_final = create_perc(df_final, 0.005)
            df_final = create_stats(df_final, dias_analisados = 30,horario = hora)
            resultado = calcular_drawdown(df_final, hora)
            lista_resultados.append(resultado)
        except:
            continue

df_resultado = pd.DataFrame(lista_resultados)

Baixando os dados de SEQL3
Baixando os dados de AERI3
Baixando os dados de WEST3
Baixando os dados de TRAD3
Baixando os dados de KRSA3
Baixando os dados de ESPA3
Baixando os dados de IFCM3
Baixando os dados de NGRD3
Baixando os dados de VIIA3
Baixando os dados de MEAL3
Baixando os dados de SGPS3
Baixando os dados de BMGB4
Baixando os dados de LPSB3
Baixando os dados de PDTC3
Baixando os dados de SHOW3
Baixando os dados de CVCB3
Baixando os dados de LUPA3
Baixando os dados de MGLU3
Baixando os dados de UCAS3
Baixando os dados de BRIT3
Baixando os dados de MLAS3
Baixando os dados de COGN3
Baixando os dados de TECN3
Baixando os dados de PINE4
Baixando os dados de NINJ3
Baixando os dados de TCSA3
Baixando os dados de MBLY3
Baixando os dados de HBSA3
Baixando os dados de RAIZ4
Baixando os dados de CIEL3
Baixando os dados de HBOR3
Baixando os dados de CMIN3
Baixando os dados de SYNE3
Baixando os dados de PGMN3
Baixando os dados de QUAL3
Baixando os dados de PFRM3
Baixando os dados de ZAMP3
B

In [12]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] >= 0.75) 
                    & (df_resultado['Pior Dia'] > -1.5) 
                    & (df_resultado['Media Geral'] > 0.3)
                    & (df_resultado['Quantidade de Trades'] >= 20)
                    ].sort_values(['Media Geral'], ascending=False)
selecionados

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
1138,BRSR6,5,19.142476,2.076125,-0.975610,2023-09-01,2023-08-23,30,0.800000,0.638083,0.898380,-0.403109,0.638083,13.619693,-0.975610,2023-09-01
1137,BRSR6,4,18.498499,2.392344,-1.021277,2023-11-28,2023-09-01,30,0.766667,0.616617,0.957270,-0.502673,0.616617,11.396419,-1.856698,2023-11-29
1139,BRSR6,6,13.585331,1.607717,-0.975610,2023-09-01,2023-08-22,30,0.766667,0.452844,0.706989,-0.382202,0.452844,10.809717,-0.975610,2023-09-01
1323,NEOE3,5,10.154291,1.455301,-1.034295,2023-10-16,2023-07-21,30,0.766667,0.338476,0.551549,-0.361619,0.338476,9.604232,-1.254559,2023-10-16
1488,KLBN11,7,9.640151,0.867679,-0.839593,2023-09-01,2023-08-28,30,0.866667,0.321338,0.434496,-0.414189,0.321338,12.955846,-0.839593,2023-09-01
1325,NEOE3,7,9.034829,1.348436,-0.596206,2023-10-16,2023-08-04,30,0.766667,0.301161,0.518167,-0.411858,0.301161,9.320087,-1.044310,2023-08-29


2 - 11:00

3 - 12:00

4 - 13:00

5 - 14:00

6 - 15:00

7 - 16:00

8 - 17:00

Comprar a abertura do horário acima

## Vender com - 0.5%

In [14]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] <= 0.25) 
                    & (df_resultado['Melhor dia'] < 1.5) 
                    & (df_resultado['Media Geral'] < -0.3)
                    & (df_resultado['Quantidade de Trades'] >= 20)
                    ].sort_values(['Media Geral'], ascending=False)
selecionados

,ativo,horário,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo,Data do Drawdown
223,RAIZ4,8,-10.489623,0.813008,-3.012048,2023-01-11,2022-12-14,30,0.200000,-0.349654,0.532510,-0.570195,-0.349654,-7.688158,-11.928168,2023-11-20
542,USIM3,8,-12.648210,1.002865,-3.094233,2022-12-14,2022-12-20,30,0.233333,-0.421607,0.550866,-0.717577,-0.421607,-7.688710,-10.269540,2023-11-27
